https://docs.llamaindex.ai/en/stable/examples/low_level/oss_ingestion_retrieval.html

- Sentence Transformers as the embedding model

- Postgres as the vector store (we support many other vector stores too!)

- Llama 2 as the LLM (through llama.cpp)

In [1]:
import configparser

config = configparser.ConfigParser()
config.read('../env/pinecone.conf')

api_key        = config["DEFAULT"]["PINECONE_API_KEY"]
environment    = config["DEFAULT"]["PINECONE_ENVIRONMENT"]
openai_api_key = config["DEFAULT"]["OPENAI_API_KEY"]

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding   # https://docs.llamaindex.ai/en/latest/examples/embeddings/OpenAI.html

# huggingface
#embed_model = HuggingFaceEmbedding(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")
embed_model = HuggingFaceEmbedding(model_name="kakaobank/kf-deberta-base")
#embed_model = HuggingFaceEmbedding(model_name="intfloat/e5-large-v2")
#embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")        # sota ?

# openai
import os
os.environ["OPENAI_API_KEY"] = openai_api_key

#embed_model = OpenAIEmbedding(model="text-embedding-3-small")


/Users/parkhyerin/Study/TIL_RAG/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
from llama_index.llms.llama_cpp import LlamaCPP

model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"
#model_url = "https://huggingface.co/TheBloke/SOLAR-10.7B-Instruct-v1.0-GGUF/resolve/main/solar-10.7b-instruct-v1.0.Q4_0.gguf"
#model_url = "https://huggingface.co/davidkim205/komt-mistral-7b-v1-gguf/resolve/main/ggml-model-q4_0.gguf"
#model_url = "https://huggingface.co/davidkim205/komt-Llama-2-13b-hf-ggml/resolve/main/ggml-model-q4_0.bin"  # gguf_init_from_file: invalid magic characters 'tjgg'
#model_url = "https://huggingface.co/StarFox7/Llama-2-ko-7B-chat-gguf/resolve/main/Llama-2-ko-7B-chat-gguf-q4_0.bin" # gguf_init_from_file: GGUFv1 is no longer supported. please use a more up-to-date version

llm = LlamaCPP(
    # You can pass in the URL to a GGUF model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/parkhyerin/Library/Caches/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:            

### Initialize Postgres

In [4]:
import psycopg2

db_name = "vector_db"
host = "localhost"
password = "1234"
port = "5432"
user = "local"

conn = psycopg2.connect(
    dbname="postgres",
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS {db_name}")
    c.execute(f"CREATE DATABASE {db_name}")

In [5]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="table",
    embed_dim=768,  # sbert, kakaobank
    #embed_dim=1536,  # openai embedding dimension
    #embed_dim=1024,  # bge-m3 embedding dimension
)

### Build an Ingestion Pipeline from Scratch

#### 1. Load Data

In [6]:
# read pdf 

from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

#loader = PyMuPDFReader()
#documents = loader.load(file_path="../data/sample.pdf")

In [7]:
# read data directory
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
# https://docs.llamaindex.ai/en/stable/examples/data_connectors/simple_directory_reader.html

required_exts = [".txt"]

reader = SimpleDirectoryReader(
    input_dir="../data",
    required_exts=required_exts,
    recursive=True,
)

documents = reader.load_data()

In [8]:
documents

[Document(id_='886d3513-80f6-4f46-80cd-576d2b26f35a', embedding=None, metadata={'file_path': '../data/1.txt', 'file_name': '1.txt', 'file_type': 'text/plain', 'file_size': 3465, 'creation_date': '2024-02-23', 'last_modified_date': '2024-02-23', 'last_accessed_date': '2024-02-23'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='생성형 인공지능(AI) 업계에서 오픈AI와 패권을 다투는 구글이 ‘오픈소스(개방형) AI’ 진영으로 한 걸음 이동했다. 업계는 챗GPT의 기반 모델인 ‘GPT4’의 어떤 것도 공개하지 않는 오픈AI와 구글의 기술 경쟁이 어떤 결말을 가져올지 주목하고 있다.\n\n구글은 21일(현지시간) 거대언어모델(LLM) ‘젬마’를 오픈소스로 공개했다. 이에 따라 개별 연구자나 개발자, 기업, 연구기관 등은 젬마를 자유롭게 활용할 수 있다. 젬마는 구글의 AI 모델 ‘제미나이’의 경량형이라고 생각하면 쉽다. AI 모델의 규모를 판단하는 매개변수가 200억개, 700억개인 두 개의 모델로 공개됐다. 제미나이의 매개변수는 1조 7800억개 이상으로 추정된다.\n\n오픈소스는 AI 이외의 분야에서도 정보기술(IT) 개발자들이 널리 공감하는 개념이다. AI 모델을 비롯한 컴퓨터 프로그램의

#### 2. Use a Text Splitter to Split Documents

In [9]:
from llama_index.core.node_parser import SentenceSplitter

In [10]:
text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [11]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

#### 3. Manually Construct Nodes from Text Chunks

In [12]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

#### 4. Generate Embeddings for each Node

In [13]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

#### 5. Load Nodes into a Vector Store

In [14]:
vector_store.add(nodes)

['66f11f29-7609-4aec-a8c8-9c371ac92408',
 '175294ce-92e5-4fe1-8bfb-7a491d537d84',
 '8567195a-c4b3-48e5-add1-c7000d5e2c60']

### Build Retrieval Pipeline from Scratch

In [15]:
query_str = "젬마가 뭔가요?"

#### 1. Generate a Query Embedding

In [16]:
query_embedding = embed_model.get_query_embedding(query_str)

#### 2. Query the Vector Database

In [17]:
# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [18]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())

현재까지의 초전도체는 영하 269도부터 영하 183도까지라는 극저온 환경에서 초전도 현상을 보였습니다. 2019년에는 영하 23도, 압력 1700억 Pa에서 초전도 현상을 보이는 란타늄 수소 화합물 만들기도 했습니다. 이때의 환경은 초고압 환경입니다. 1기압이 101325 Pa이니까 상상할 수 없는 높은 압력입니다.만약 상온 상압에서도 초전도체로 동작하는 물질이 생긴다면 에너지 혁명이 일어날 것입니다. 상온 상압 초전도체를 개발한 사람은 의심할 것 없이 노벨상입니다. 그러므로 2023년 7월에 대한민국에서 상온, 상압의 초전도체를 만들었다는 뉴스는 충분히 세상을 놀라게 할 만합니다. 그 초전도체가 바로 LK-99입니다.

퀀텀에너지 연구소는 논문과 함께 초전도체라고 주장하는 LK-99의 영상을 공개했습니다. 자석 위에 동전 모양의 검은 물체가 비스듬히 들려 있습니다. 완전 공중부양이 아니라 한쪽 면이 자석위에 살짝 닿아있는데요. 어쨌거나 뜨다시피 한 모습은 놀랍습니다. 이 물질이 바로 초전도체 물질이라고 주장하는 LK-99입니다.2028년 8월 현재 세계의 유명한 대학과 연구소들은 상온 상압 초전도체인 LK-99를 검증하고 있는 중입니다. 예를들어 미국의 프린스턴 대학, 독일의 막스플랑크 연구소는 LK-99가 특이한 성질을 보였지만 초전도체는 아닌듯하다는 결론을 내놓았습니다. LK-99가 초전도체가 아니다라고 단정한 것은 아닙니다. 다만 연구자들을 설득할 만한 충분한 데이터와 설명이 부족하다는 것입니다. 또 다른 과학자들의 검증결과가 기다려집니다.

2023년 7월 LK-99라는 초전도체에 대한 논문이 아카이브에 올라왔고, 그 이후로 초전도체 뉴스가 뜨겁습니다. LK-99라는 초전도체 개발 가능성은 초전도체 활용 분야로 관심이 갔고, 즉시 주식시장에 그 관심이 반영되었습니다.올해 시장 최대 테마주는 초전도체 관련 주입니다. 초전도체 응용기술을 가졌다는 어떤 기업은 2023년 7월31일부터 상한가 3회로 사흘만에 2배가 올랐습니다. 거래정지 하루가 있었고, 그 

### 3. Parse Result into a Set of Nodes

In [19]:
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

### 4. Put into a Retriever

In [20]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [21]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

### Plug this into our RetrieverQueryEngine to synthesize a response

In [22]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [23]:
#query_str = "How does Llama 2 perform compared to other open-source models?"
query_str = "젬마가 무엇인가요?"

response = query_engine.query(query_str)


llama_print_timings:        load time =   14379.46 ms
llama_print_timings:      sample time =       2.74 ms /    37 runs   (    0.07 ms per token, 13513.51 tokens per second)
llama_print_timings: prompt eval time =  110117.09 ms /  3233 tokens (   34.06 ms per token,    29.36 tokens per second)
llama_print_timings:        eval time =    4004.33 ms /    36 runs   (  111.23 ms per token,     8.99 tokens per second)
llama_print_timings:       total time =  114188.84 ms /  3269 tokens


In [24]:
print(str(response))

젬마는 구글의 생성형 AI 모델(LLM)입니다.


In [25]:
print(response.source_nodes[0].get_content())

현재까지의 초전도체는 영하 269도부터 영하 183도까지라는 극저온 환경에서 초전도 현상을 보였습니다. 2019년에는 영하 23도, 압력 1700억 Pa에서 초전도 현상을 보이는 란타늄 수소 화합물 만들기도 했습니다. 이때의 환경은 초고압 환경입니다. 1기압이 101325 Pa이니까 상상할 수 없는 높은 압력입니다.만약 상온 상압에서도 초전도체로 동작하는 물질이 생긴다면 에너지 혁명이 일어날 것입니다. 상온 상압 초전도체를 개발한 사람은 의심할 것 없이 노벨상입니다. 그러므로 2023년 7월에 대한민국에서 상온, 상압의 초전도체를 만들었다는 뉴스는 충분히 세상을 놀라게 할 만합니다. 그 초전도체가 바로 LK-99입니다.

퀀텀에너지 연구소는 논문과 함께 초전도체라고 주장하는 LK-99의 영상을 공개했습니다. 자석 위에 동전 모양의 검은 물체가 비스듬히 들려 있습니다. 완전 공중부양이 아니라 한쪽 면이 자석위에 살짝 닿아있는데요. 어쨌거나 뜨다시피 한 모습은 놀랍습니다. 이 물질이 바로 초전도체 물질이라고 주장하는 LK-99입니다.2028년 8월 현재 세계의 유명한 대학과 연구소들은 상온 상압 초전도체인 LK-99를 검증하고 있는 중입니다. 예를들어 미국의 프린스턴 대학, 독일의 막스플랑크 연구소는 LK-99가 특이한 성질을 보였지만 초전도체는 아닌듯하다는 결론을 내놓았습니다. LK-99가 초전도체가 아니다라고 단정한 것은 아닙니다. 다만 연구자들을 설득할 만한 충분한 데이터와 설명이 부족하다는 것입니다. 또 다른 과학자들의 검증결과가 기다려집니다.

2023년 7월 LK-99라는 초전도체에 대한 논문이 아카이브에 올라왔고, 그 이후로 초전도체 뉴스가 뜨겁습니다. LK-99라는 초전도체 개발 가능성은 초전도체 활용 분야로 관심이 갔고, 즉시 주식시장에 그 관심이 반영되었습니다.올해 시장 최대 테마주는 초전도체 관련 주입니다. 초전도체 응용기술을 가졌다는 어떤 기업은 2023년 7월31일부터 상한가 3회로 사흘만에 2배가 올랐습니다. 거래정지 하루가 있었고, 그 

In [26]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='8567195a-c4b3-48e5-add1-c7000d5e2c60', embedding=None, metadata={'file_path': '../data/2.txt', 'file_name': '2.txt', 'file_type': 'text/plain', 'file_size': 2495, 'creation_date': '2024-02-23', 'last_modified_date': '2024-02-23', 'last_accessed_date': '2024-02-23'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='현재까지의 초전도체는 영하 269도부터 영하 183도까지라는 극저온 환경에서 초전도 현상을 보였습니다. 2019년에는 영하 23도, 압력 1700억 Pa에서 초전도 현상을 보이는 란타늄 수소 화합물 만들기도 했습니다. 이때의 환경은 초고압 환경입니다. 1기압이 101325 Pa이니까 상상할 수 없는 높은 압력입니다.만약 상온 상압에서도 초전도체로 동작하는 물질이 생긴다면 에너지 혁명이 일어날 것입니다. 상온 상압 초전도체를 개발한 사람은 의심할 것 없이 노벨상입니다. 그러므로 2023년 7월에 대한민국에서 상온, 상압의 초전도체를 만들었다는 뉴스는 충분히 세상을 놀라게 할 만합니다. 그 초전도체가 바로 LK-99입니다.\n\n퀀텀에너지 연구소는 논문과 함께 초전도체라고 주장하는 LK-99의 영상을 공개했습니다. 자석 위에 동전 모양의 검은 물체가 비스듬히 들려 있습니다. 완전 공중부양이 아니라 한쪽 면이 자석위에 살짝 닿아있는데요. 어쨌거나 뜨다시피 한 모습은 놀랍습니다. 이 물질이 바로 초전도체 물질이라고 주장하는 LK-99입니다.2028년 8월 현재 세계의 유명한 대학과 연구소들은 상온 상압 초전도체인 LK-99를 검증하고 있는 중입니다. 예를들어 미국의 프린스턴 대학, 독